# CWT scalograms

Building model using a reduced dataset and load it for test set predictions in Kaggle.

5 channels (LT, RT, LP, RP, C).

Implementing tf.keras.metrics.KLDivergence().

- Training run.
- Saving model and checkpoint.
- Inspecting: loading model and checkpoint.
- Preprocessing of test eegs.
- Predictions and submission.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import regularizers
# import matplotlib.pyplot as plt

base_dir = '../../kaggle_data/hms'
# base_dir = '../../data/hms'
# base_dir = '/kaggle/input/hms-harmful-brain-activity-classification'

devset_dir = '../data'
# devset_dir = '/kaggle/input/hms-single-spectrograms-v1'

# path_train = f'{devset_dir}/05_single_cwt_v1_train.npy'
# path_train_items = f'{devset_dir}/05_single_cwt_v1_train_items.npy'
# path_val = f'{devset_dir}/05_single_cwt_v1_val.npy'
# path_val_items = f'{devset_dir}/05_single_cwt_v1_val_items.npy'
# path_test = f'{devset_dir}/05_single_cwt_v1_test.npy'
# path_test_items = f'{devset_dir}/05_single_cwt_v1_test_items.npy'

2024-03-10 16:20:45.509572: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-10 16:20:45.596400: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path_train = f'{devset_dir}/05_reduced_single_cwt_v1_train.npy'
path_train_items = f'{devset_dir}/05_reduced_single_cwt_v1_train_items.npy'
path_val = f'{devset_dir}/05_reduced_single_cwt_v1_val.npy'
path_val_items = f'{devset_dir}/05_reduced_single_cwt_v1_val_items.npy'
path_test = f'{devset_dir}/05_reduced_single_cwt_v1_test.npy'
path_test_items = f'{devset_dir}/05_reduced_single_cwt_v1_test_items.npy'

## Definitions


## Train

In [3]:
#
# Data generator using numpy and no pandas.
#
# scalograms
# 30 seconds slice (I think)
# 5 channels (LP, RP, LT, RP, C)
#

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=True):
        ''' Initialization
        item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
        seizure_vote, lpd_vote, gpd_vote, lrda_vote,
        grda_vote, other_vote]
        '''
        self.n_channels = 5
        # self.n_freqs = 40

        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))
        y = np.empty((true_size, self.n_classes), dtype=float)

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]
            # Store solution
            y[i,:] = item[-6:]

        return X, y

In [4]:

def make_model(input_shape, num_classes):
    input_layer = keras.layers.Input(input_shape)

    #max1 = keras.layers.MaxPooling1D(pool_size=2)(input_layer)
    
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.MaxPooling2D(pool_size=4)(conv1)
    conv1 = keras.layers.ReLU()(conv1)
    
    # conv2 = keras.layers.Conv2D(filters=64, kernel_size=7, padding="same")(conv1)
    # #conv2 = keras.layers.BatchNormalization()(conv2)
    # # conv2 = keras.layers.MaxPooling2D(pool_size=8)(conv2)
    # conv2 = keras.layers.ReLU()(conv2)

    # conv3 = keras.layers.Conv2D(filters=256, kernel_size=7, padding="same")(conv2)
    # #conv3 = keras.layers.BatchNormalization()(conv3)
    # conv3 = keras.layers.MaxPooling2D(pool_size=2)(conv3)
    # conv3 = keras.layers.ReLU()(conv3)

    # conv4 = keras.layers.Conv2D(filters=512, kernel_size=3, padding="same")(conv3)
    # conv4 = keras.layers.BatchNormalization()(conv4)
    # conv4 = keras.layers.MaxPooling2D(pool_size=4)(conv4)
    # conv4 = keras.layers.ReLU()(conv4)

    fltn  = keras.layers.Flatten()(conv1) 
    
    relu1 = keras.layers.Dense(64)(fltn)
    relu1 = keras.layers.ReLU()(relu1)

    # relu2 = keras.layers.Dense(64)(relu1)
    # relu2 = keras.layers.ReLU(64)(relu2)

#     lin = keras.layers.Dense(2)(relu2)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(relu1)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


In [5]:
# Parameters
params = {
    'batch_size': 32,
    'n_classes': 6,
    'shuffle': True
    }

training_generator = DataGenerator(path_train_items, path_train , **params)
validation_generator = DataGenerator(path_val_items, path_val, **params)

print("Observations in training set:", training_generator.__len__()*params['batch_size'])
print("Observations in validation set:", validation_generator.__len__()*params['batch_size'])


Observations in training set: 512
Observations in validation set: 128


In [20]:
# # Kaggle version
# # Name to monitor is different.
# checkpoint_filepath = 'results/checkpoint1.model.keras'
# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     monitor='val_kl_divergence',
#     mode='min',
#     save_best_only=True)


In [21]:
# Name to monitor is different.
checkpoint_filepath = 'results/checkpoint1.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_kullback_leibler_divergence',
    mode='min',
    save_best_only=True)


In [23]:
opt = keras.optimizers.SGD(
    learning_rate=0.01,
    momentum=0.01,
)

# opt = keras.optimizers.Adam(
#     learning_rate=0.004,
# )

dim = training_generator.get_dim()

model = make_model(input_shape=dim, num_classes=6)

# model.load_weights('/kaggle/input/hms-model-cwt-v1/checkpoint.model.keras')

model.compile(optimizer=opt,
            loss=tf.keras.losses.KLDivergence(),
            metrics=[tf.keras.metrics.KLDivergence()])

model.fit(training_generator, epochs=10,
          validation_data=validation_generator,
          callbacks=[model_checkpoint_callback])

Epoch 1/10
16/16 [==============================] - 3s 181ms/step - loss: 11.5522 - kullback_leibler_divergence: 11.5522 - val_loss: 4.4372 - val_kullback_leibler_divergence: 4.4372
Epoch 2/10
16/16 [==============================] - 3s 175ms/step - loss: 4.3561 - kullback_leibler_divergence: 4.3561 - val_loss: 1.4256 - val_kullback_leibler_divergence: 1.4256
Epoch 3/10
16/16 [==============================] - 3s 178ms/step - loss: 1.4083 - kullback_leibler_divergence: 1.4083 - val_loss: 1.4109 - val_kullback_leibler_divergence: 1.4109
Epoch 4/10
16/16 [==============================] - 3s 171ms/step - loss: 1.3545 - kullback_leibler_divergence: 1.3545 - val_loss: 1.4067 - val_kullback_leibler_divergence: 1.4067
Epoch 5/10
16/16 [==============================] - 3s 176ms/step - loss: 1.3115 - kullback_leibler_divergence: 1.3115 - val_loss: 1.4000 - val_kullback_leibler_divergence: 1.4000
Epoch 6/10
16/16 [==============================] - 3s 168ms/step - loss: 1.3335 - kullback_leible

In [24]:
model.save("results/hms-keras-10-model-reduced.keras")

## Inspecting model

In [54]:
new_model = keras.models.load_model("results/hms-keras-10-model-reduced.keras")

### Predictions

In [55]:
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

#
# Test Data generator: for predicting
# using own test set.
# (Not for predicting LB)
#

class TestDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path_to_items, path_to_data, batch_size=32, n_classes=6, shuffle=False):
        ''' Initialization
        items: [eeg_id, eeg_sub_id, idx of offset, target, ...]
        '''
        self.n_channels = 5
        self.data = np.load(path_to_data)
        self.items = np.load(path_to_items)
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        # pass 
        
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]

        return X
    
                
params = {
    'batch_size': 32,
    'n_classes': 6,
    }

test_generator = TestDataGenerator(path_test_items, path_test, **params)


In [56]:
y_pred = model.predict(test_generator)


4/4 [==============================] - 0s 39ms/step


In [57]:
y_pred_new = new_model.predict(test_generator)


4/4 [==============================] - 0s 40ms/step


In [58]:
np.all(y_pred == y_pred_new)

True

Loading the model produce the same predictions.

### Checkpoint

In [59]:
chk_model = make_model(input_shape=dim, num_classes=6)

chk_model.load_weights(checkpoint_filepath)

chk_model.compile(optimizer=opt,
            loss=tf.keras.losses.KLDivergence(),
            metrics=[tf.keras.metrics.KLDivergence()])


In [60]:
y_pred_chk = chk_model.predict(test_generator)


4/4 [==============================] - 0s 42ms/step


In [61]:
np.all(y_pred == y_pred_chk)

False

Loading model, then loading weights of checkpoint.

In [62]:
new_model.load_weights(checkpoint_filepath)


In [63]:
y_pred_new_and_chkp = new_model.predict(test_generator)


4/4 [==============================] - 0s 38ms/step


In [64]:
np.all(y_pred_chk == y_pred_new_and_chkp)

True

Same predictions.

In [34]:
chk_model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 49, 400, 5)]      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 49, 400, 32)       1472      
                                                                 
 batch_normalization_5 (Bat  (None, 49, 400, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 12, 100, 32)       0         
 g2D)                                                            
                                                                 
 re_lu_10 (ReLU)             (None, 12, 100, 32)       0         
                                                                 
 flatten_5 (Flatten)         (None, 38400)             0   

## Test eegs preprocessing for Kaggle

Things to change for Kaggle:

1. Folder for libs.
1. Folder for test_eegs
1. Interpolate?
1. Remove timing.

In [49]:
import os
import pywt
import time

import sys
sys.path.insert(0, '../lib')
from lib_banana import banana
from lib_pooling import poolingOverlap


test_path = '../toy_data/test_eegs'

test_files = os.listdir(test_path)
test_size = len(test_files)
test_size

292

In [43]:
scales = np.arange(1,50)
waveletname = 'morl'
n_channels = 5
dim1 = scales.shape[0]
pool_window = 5
dim2 = int(2000/pool_window)
sampling_period = 1
# Center 10 s adjusted by pooling window.
start2 = int(4000/pool_window)
end2 = int(6000/pool_window)

sgrams = np.empty((test_size, dim1, dim2, n_channels))
# item: [eeg_id, eeg_sub_id, idx in sgrams (1st index), target,
#       seizure_vote, lpd_vote, gpd_vote, lrda_vote,
#       grda_vote, other_vote]
items = np.array([], dtype=float).reshape(0,10)

In [50]:
t1 = time.perf_counter()

for i, file in enumerate(test_files):
    eeg_full = pd.read_parquet(f'{test_path}/{file}')
    # eeg_full = eeg_full.interpolate(limit_direction='both') # <<<<< Interpolation
    eeg = banana(eeg_full, filter=False)

        # Averaging each chain in the banana montage.

    # Left temporal chain.
    coeff = np.zeros((dim1, 10000))
    # coeff = np.zeros((dim1, 6000))  # keeping 30 s to reduce runtime.
    for col in [0,1,2,3]:
        coeff_, freq = pywt.cwt(eeg.iloc[:,col], scales, waveletname, sampling_period=sampling_period)
        coeff = coeff + coeff_

    coeff = coeff/4
    coeff = poolingOverlap(coeff,(1,pool_window),stride=None,method='mean',pad=False)
    coeff = (coeff - np.mean(coeff)) / np.std(coeff)
    sgrams[i,:,:,0] = coeff[:,start2:end2].copy()

    # Right temporal chain.
    coeff = np.zeros((dim1, 10000))
    for col in [4,5,6,7]:
        coeff_, freq = pywt.cwt(eeg.iloc[:,col], scales, waveletname, sampling_period=sampling_period)
        coeff = coeff + coeff_

    coeff = coeff/4
    coeff = poolingOverlap(coeff,(1,pool_window),stride=None,method='mean',pad=False)
    coeff = (coeff - np.mean(coeff)) / np.std(coeff)
    sgrams[i,:,:,1] = coeff[:,start2:end2].copy()

    # Left parasagittal chain.
    coeff = np.zeros((dim1, 10000))
    for col in [8,9,10,11]:
        coeff_, freq = pywt.cwt(eeg.iloc[:,col], scales, waveletname, sampling_period=sampling_period)
        coeff = coeff + coeff_

    coeff = coeff/4
    coeff = poolingOverlap(coeff,(1,pool_window),stride=None,method='mean',pad=False)
    coeff = (coeff - np.mean(coeff)) / np.std(coeff)
    sgrams[i,:,:,2] = coeff[:,start2:end2].copy()

    # Right parasagittal chain.
    coeff = np.zeros((dim1, 10000))
    for col in [12,13,14,15]:
        coeff_, freq = pywt.cwt(eeg.iloc[:,col], scales, waveletname, sampling_period=sampling_period)
        coeff = coeff + coeff_

    coeff = coeff/4
    coeff = poolingOverlap(coeff,(1,pool_window),stride=None,method='mean',pad=False)
    coeff = (coeff - np.mean(coeff)) / np.std(coeff)
    sgrams[i,:,:,3] = coeff[:,start2:end2].copy()

    # Central chain.
    coeff = np.zeros((dim1, 10000))
    for col in [16,17]:
        coeff_, freq = pywt.cwt(eeg.iloc[:,col], scales, waveletname, sampling_period=sampling_period)
        coeff = coeff + coeff_

    coeff = coeff/2
    coeff = poolingOverlap(coeff,(1,pool_window),stride=None,method='mean',pad=False)
    coeff = (coeff - np.mean(coeff)) / np.std(coeff)
    sgrams[i,:,:,4] = coeff[:,start2:end2].copy()

    eeg_id = int(test_files[2].split('.')[0])
    # Set unkowns to zero, to reuse code.
    xitem = np.array([eeg_id, 0, i, 0, 0, 0, 0,
                      0, 0, 0], dtype=float).reshape(1,10)
    items = np.concatenate([items, xitem])

t2 = time.perf_counter()
print(f'Time for preprocessing {test_size} files: {np.round(t2-t1,3)} s.')

Time for preprocessing 292 files: 78.77589142999932 s.


In [53]:
t1 = time.perf_counter()

TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

#
# Test Data generator
#
# for predictions in Kaggle.
# 

class TestDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, items, data, batch_size=32, n_classes=6, shuffle=False):
        ''' Initialization
        items: [eeg_id, eeg_sub_id, idx of offset, target, ...]
        '''
        self.n_channels = 5
        self.data = data
        self.items = items
        self.dim = (self.data.shape[1], self.data.shape[2])
        self.batch_size = batch_size
        self.len = self.data.shape[0]
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def get_dim(self):
        'Dimensions for the input layer.'
        return (self.dim[0], self.dim[1], self.n_channels)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.len)
        # pass 
        
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        true_size = len(indexes)
        X = np.empty((true_size, *self.dim, self.n_channels))

        # Generate data
        for i, idx in enumerate(indexes):
            item = self.items[idx]
            # print(item)  # Uncomment for testing.
            X[i,:,:,:] = self.data[np.int32(item[2]), :, :, :]

        return X

params = {
    'batch_size': 32,
    'n_classes': 6,
    }

test_generator = TestDataGenerator(items, sgrams, **params)


Loading the model and predict.

In [65]:
loaded_model = keras.models.load_model("results/hms-keras-10-model-reduced.keras")
loaded_model.load_weights(checkpoint_filepath)
y_pred = loaded_model.predict(test_generator)



4/4 [==============================] - 0s 39ms/step


In [66]:
t2 = time.perf_counter()
print(f'Time for predicting {test_size} files: {np.round(t2-t1,3)} s.')

Time for predicting 292 files: 782.806 s.
